In [8]:
import pandas as pd

In [2]:
df_travel = pd.read_csv("travel-times.csv")
df_travel

,Date,StartTime,DayOfWeek,GoingTo,Distance,MaxSpeed,AvgSpeed,AvgMovingSpeed,FuelEconomy,TotalTime,MovingTime,Take407All,Comments
0,1/6/2012,16:37,Friday,Home,51.29,127.4,78.3,84.8,NaN,39.3,36.3,No,NaN
1,1/6/2012,08:20,Friday,GSK,51.63,130.3,81.8,88.9,NaN,37.9,34.9,No,NaN
2,1/4/2012,16:17,Wednesday,Home,51.27,127.4,82.0,85.8,NaN,37.5,35.9,No,NaN
3,1/4/2012,07:53,Wednesday,GSK,49.17,132.3,74.2,82.9,NaN,39.8,35.6,No,NaN
4,1/3/2012,18:57,Tuesday,Home,51.15,136.2,83.4,88.1,NaN,36.8,34.8,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,7/18/2011,08:09,Monday,GSK,54.52,125.6,49.9,82.4,7.89,65.5,39.7,No,NaN
201,7/14/2011,08:03,Thursday,GSK,50.90,123.7,76.2,95.1,7.89,40.1,32.1,Yes,NaN
202,7/13/2011,17:08,Wednesday,Home,51.96,132.6,57.5,76.7,NaN,54.2,40.6,Yes,NaN
203,7/12/2011,17:51,Tuesday,Home,53.28,125.8,61.6,87.6,NaN,51.9,36.5,Yes,NaN


- Calcula o endtime (StartTime + TotalTime).

In [62]:
df_work = df_travel.copy()
df_work['Date'] = pd.DatetimeIndex(df_travel['Date'])

# Parseo de StartTime a datetime
df_work['StartTime'] = pd.to_datetime(df_travel['StartTime'], format="%H:%M")

# Cálculo del EndTime y conversión a formato horas:minutos
df_work['EndTime'] = (df_work['StartTime'] + pd.TimedeltaIndex(df_work['TotalTime'], unit="m")).dt.strftime('%H:%M')

# Convertir StartTime a formato horas:minutos
df_work['StartTime'] = (df_work['StartTime']).dt.strftime("%H:%M")

df_work[['StartTime','TotalTime','EndTime']]

,StartTime,TotalTime,EndTime
0,16:37,39.3,17:16
1,08:20,37.9,08:57
2,16:17,37.5,16:54
3,07:53,39.8,08:32
4,18:57,36.8,19:33
...,...,...,...
200,08:09,65.5,09:14
201,08:03,40.1,08:43
202,17:08,54.2,18:02
203,17:51,51.9,18:42


- Agrupa por día da semana e indica a media que lle leva chegar (TotalTime) por día da semana.

In [64]:
media_por_dia = df_work.groupby("DayOfWeek")["TotalTime"].mean()
df_media_dia = pd.DataFrame(media_por_dia)
df_media_dia

,TotalTime
DayOfWeek,
Friday,37.922222
Monday,43.197436
Thursday,41.177273
Tuesday,42.520833
Wednesday,43.170213


- Fai un delta (diferencia) entre o TotalTime e o MovingTime. Indica cal foi o día con maior diferencia e o de menor diferencia.

Este foi o día con maior diferencia:

In [96]:
# Creación do delta
df_work['delta'] = (df_work[['MovingTime', 'TotalTime']].diff(axis=1))['TotalTime']

df_work[df_work['delta'] == df_work['delta'].max()]

,Date,StartTime,DayOfWeek,GoingTo,Distance,MaxSpeed,AvgSpeed,AvgMovingSpeed,FuelEconomy,TotalTime,MovingTime,Take407All,Comments,EndTime,delta
200,2011-07-18,08:09,Monday,GSK,54.52,125.6,49.9,82.4,7.89,65.5,39.7,No,NaN,09:14,25.8


E este o de menos diferencia:

In [97]:
df_work[df_work['delta'] == df_work['delta'].min()]

,Date,StartTime,DayOfWeek,GoingTo,Distance,MaxSpeed,AvgSpeed,AvgMovingSpeed,FuelEconomy,TotalTime,MovingTime,Take407All,Comments,EndTime,delta
149,2011-08-24,16:47,Wednesday,Home,51.01,132.9,79.6,86.1,8.54,38.5,38.5,No,NaN,17:25,0.0


- Agrupa os datos por día da semana collendo a media de tempo (o delta do apartado anterior) e indica o día da semana de menor e maior diferencia.

In [158]:
delta_por_dia = df_work.groupby("DayOfWeek")["delta"].mean()
delta_por_dia = pd.DataFrame(delta_por_dia)
delta_por_dia

,delta
DayOfWeek,
Friday,2.807407
Monday,5.051282
Thursday,3.759091
Tuesday,4.093750
Wednesday,4.085106


Día con maior diferencia:

In [162]:
delta_por_dia.loc[delta_por_dia['delta'] == delta_por_dia['delta'].max()]

,delta
DayOfWeek,
Monday,5.051282


Día con menor diferencia:

In [163]:
delta_por_dia.loc[delta_por_dia['delta'] == delta_por_dia['delta'].min()]

,delta
DayOfWeek,
Friday,2.807407


- Elimina as columnas na que máis da metade dos valores sexan nulos.

O argumento **thresh** especifica o número mínimo de valores non nulos que debe ter unha fila ou columna para non ser eliminada.

In [165]:
df_work = df_work.dropna(thresh=int(0.5 * len(df_work)), axis=1)
df_work.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            205 non-null    datetime64[ns]
 1   StartTime       205 non-null    object        
 2   DayOfWeek       205 non-null    object        
 3   GoingTo         205 non-null    object        
 4   Distance        205 non-null    float64       
 5   MaxSpeed        205 non-null    float64       
 6   AvgSpeed        205 non-null    float64       
 7   AvgMovingSpeed  205 non-null    float64       
 8   FuelEconomy     188 non-null    object        
 9   TotalTime       205 non-null    float64       
 10  MovingTime      205 non-null    float64       
 11  Take407All      205 non-null    object        
 12  EndTime         205 non-null    object        
 13  delta           205 non-null    float64       
dtypes: datetime64[ns](1), float64(7), object(6)
memory usage: 

- Visualiza o número de nulos de cada columna.

In [173]:
columnas = df_work.columns.to_list()
for i in columnas:
    print(f"{i} -> {df_work[i].isnull().sum()} valores nulos")

Date -> 0 valores nulos
StartTime -> 0 valores nulos
DayOfWeek -> 0 valores nulos
GoingTo -> 0 valores nulos
Distance -> 0 valores nulos
MaxSpeed -> 0 valores nulos
AvgSpeed -> 0 valores nulos
AvgMovingSpeed -> 0 valores nulos
FuelEconomy -> 17 valores nulos
TotalTime -> 0 valores nulos
MovingTime -> 0 valores nulos
Take407All -> 0 valores nulos
EndTime -> 0 valores nulos
delta -> 0 valores nulos


- Substitúe os valores nules empregando o valor da fila anterior (method='ffil').

In [175]:
df_work = df_work.fillna(method='ffill')

- Os primeiros valores son NaN, debemos corrixilo substituíndo estes valores polos anteriores (method='bfill').

In [183]:
df_work = df_work.fillna(method='bfill')
df_work

,Date,StartTime,DayOfWeek,GoingTo,Distance,MaxSpeed,AvgSpeed,AvgMovingSpeed,FuelEconomy,TotalTime,MovingTime,Take407All,EndTime,delta
0,2012-01-06,16:37,Friday,Home,51.29,127.4,78.3,84.8,-,39.3,36.3,No,17:16,3.0
1,2012-01-06,08:20,Friday,GSK,51.63,130.3,81.8,88.9,-,37.9,34.9,No,08:57,3.0
2,2012-01-04,16:17,Wednesday,Home,51.27,127.4,82.0,85.8,-,37.5,35.9,No,16:54,1.6
3,2012-01-04,07:53,Wednesday,GSK,49.17,132.3,74.2,82.9,-,39.8,35.6,No,08:32,4.2
4,2012-01-03,18:57,Tuesday,Home,51.15,136.2,83.4,88.1,-,36.8,34.8,No,19:33,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2011-07-18,08:09,Monday,GSK,54.52,125.6,49.9,82.4,7.89,65.5,39.7,No,09:14,25.8
201,2011-07-14,08:03,Thursday,GSK,50.90,123.7,76.2,95.1,7.89,40.1,32.1,Yes,08:43,8.0
202,2011-07-13,17:08,Wednesday,Home,51.96,132.6,57.5,76.7,7.89,54.2,40.6,Yes,18:02,13.6
203,2011-07-12,17:51,Tuesday,Home,53.28,125.8,61.6,87.6,7.89,51.9,36.5,Yes,18:42,15.4
